In [6]:
import sys, os
helper_py_location = os.path.abspath(os.path.join(os.path.join(os.getcwd(), "../..") ,"src","helper"))
print(helper_py_location)
sys.path.append(helper_py_location)
sys.path

e:\zhaoj\download\workSpace\Portfolio_Management\src\helper


['C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip',
 'C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312\\DLLs',
 'C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312\\Lib',
 'C:\\Users\\zhaoj\\AppData\\Local\\Programs\\Python\\Python312',
 'e:\\zhaoj\\download\\workSpace\\.venv',
 '',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages\\win32',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages\\win32\\lib',
 'e:\\zhaoj\\download\\workSpace\\.venv\\Lib\\site-packages\\Pythonwin',
 'e:\\zhaoj\\download\\workSpace\\Portfolio_Management\\src\\helper',
 'e:\\zhaoj\\download\\workSpace\\Portfolio_Management\\src\\helper']

In [7]:
import helper
HF_returns_original = helper.load_data('HW_Hedge_Fund.xlsx',1)
M_ER_country_original = helper.load_data('HW_World.xlsx')
Emp_factors_original = helper.load_data('HW_Factors.xlsx')
HP_DJ_Sto_original = helper.load_data('HW_DJIA_Prices.xlsx',4)

#clean
HF_returns =HF_returns_original.copy()
Emp_factors = Emp_factors_original.copy()
M_ER_country = M_ER_country_original.copy()
HP_DJ_Sto = HP_DJ_Sto_original.copy()

Emp_factors.iloc[:, 1:] = Emp_factors.iloc[:, 1:].div(100)
M_ER_country.iloc[:,1:] = M_ER_country.iloc[:,1:].div(100)
Emp_factors.rename(columns={Emp_factors.keys()[0]: 'Date'}, inplace=True)

### Part 3 Portfolio Construction
* famlize the Dow Jones
* famlize diff weighting methodologies

1. Different Portfolios - weights

In [8]:
print(type(HP_DJ_Sto))
HP_DJ_Sto.head()

<class 'pandas.core.frame.DataFrame'>


,Date,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,...,NKE,PG,CRM,BAC,UNH,VZ,HWP,PFE,WMT,DJIA
0,2004-06-30,90.01,51.38,54.57,49.0842,51.09,79.44,94.11,23.70,50.48,...,75.75,54.44,16.07,84.62,62.25,36.19,21.10,34.28,52.76,10139.71
1,2004-07-30,82.36,50.25,56.88,52.3474,50.75,73.49,95.65,20.92,43.86,...,72.71,52.15,13.01,85.01,62.90,38.54,20.15,31.96,53.01,10173.92
2,2004-08-31,82.36,50.02,59.29,58.8130,52.22,72.70,97.50,18.76,44.71,...,75.31,55.97,13.00,44.98,66.13,39.25,17.89,32.67,52.67,10080.27
3,2004-09-30,79.97,51.46,56.81,79.5303,51.62,80.45,53.64,18.10,40.05,...,78.80,54.12,15.63,43.33,73.74,39.38,18.75,30.60,53.20,10027.47
4,2004-10-29,77.57,53.07,56.80,101.7654,49.90,80.54,53.06,19.21,40.66,...,81.31,51.18,20.32,44.79,72.40,39.10,18.66,28.95,53.92,10428.02


Build diff weight stragy

In [9]:
import pandas as pd
import numpy as np
Portfolio = []
intern = []
# new_weight = pd.DataFrame()
number_of_stock = HP_DJ_Sto.iloc[0,1:-1].shape[0]

#value
for i in range(1,HP_DJ_Sto.shape[1]-1):
    intern.append(HP_DJ_Sto.iloc[0,i]/HP_DJ_Sto.iloc[0,1:-1].sum())
Portfolio.append(intern) 

#equal_weight
intern = [1/number_of_stock for _ in range(number_of_stock)]
Portfolio.append(intern) 

#min_var
cov_matrix = HP_DJ_Sto.iloc[:, 1:-1].cov()
one = np.ones(cov_matrix.shape[0])
Inv_cov = np.linalg.inv(cov_matrix)

# min_var_weights = Inv_cov.dot(one) / np.dot(one.T, Inv_cov.dot(one))
min_var_weights = np.dot(Inv_cov, one) / \
                  np.dot(one.T, Inv_cov.dot(one))
# min_var_portfolio = np.dot(min_var_weights.T, np.dot(cov_matrix, min_var_weights))
# print(min_var_portfolio)
Portfolio.append(min_var_weights.tolist())

#naive risk parity
vol = HP_DJ_Sto.iloc[:,1:-1].pct_change().std()*np.sqrt(12)
inverse_vol = 1 / vol
normalized_I_vol = [x / sum(inverse_vol) for x in inverse_vol]
Portfolio.append(normalized_I_vol)

#Tangent --- ost Diversified Portfolio
ret_matix = HP_DJ_Sto.iloc[:,1:-1].pct_change().mean()
tangent_weight = np.dot(Inv_cov,ret_matix)/ \
                np.dot(one.T,Inv_cov.dot(ret_matix))
Portfolio.append(tangent_weight)

# display(weight)


In [10]:
# HP_DJ_Sto.iloc[:,1:-1].pct_change()
# one = np.ones(cov_matrix.shape[0])
# ret_matix = HP_DJ_Sto.iloc[:,1:-1].pct_change().mean()
# HP_DJ_Sto.iloc[:,1:-1]

In [11]:
#buy largest
HP_DJ_Sto_Ind_ori = helper.load_data('HW_DJIA_Prices.xlsx')
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind_ori.copy()
industries = HP_DJ_Sto_Ind.iloc[0,1:-1]
# Drop the first row to retain only stock prices
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind.iloc[[4]]
largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax()
# largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax(axis = 1)
weight_of_buylarge = 1/largest_stocks.shape[1]
largest_stocks_weights=[]

for stock in HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns:
    if stock in largest_stocks.iloc[0].values:
        largest_stocks_weights.append(weight_of_buylarge)
    else:
            largest_stocks_weights.append(0)
Portfolio.append(largest_stocks_weights)

C:\Users\zhaoj\AppData\Local\Temp\ipykernel_31572\2927928264.py:7: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax()


In [12]:
# for stock in HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns:
#     print(stock)
# print("\n")
# for i in largest_stocks.iloc[0]:
#     print(i)
# largest_stocks.iloc[[0]]

In [13]:
#idustry_equal
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind_ori.copy()
industries = HP_DJ_Sto_Ind.iloc[0,1:-1]
# HP_DJ_Sto_Ind = HP_DJ_Sto_Ind.iloc[1]

number_ind = HP_DJ_Sto_Ind.groupby(industries, axis=1).count().iloc[0]
equal_weights = number_ind.loc[HP_DJ_Sto_Ind_ori.iloc[0,1:-1]].apply(lambda x:1/x*0.2).values
Portfolio.append(equal_weights)

C:\Users\zhaoj\AppData\Local\Temp\ipykernel_31572\1025226775.py:6: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  number_ind = HP_DJ_Sto_Ind.groupby(industries, axis=1).count().iloc[0]


In [14]:
# number_ind.loc[HP_DJ_Sto_Ind_ori.iloc[0,1:-1]].apply(lambda x:1/x*0.2).values


In [15]:
# weight_DF = pd.DataFrame(Portfolio,index = ['value','equal_weight','min_var','naive','tangent','large5','idustry_equal'], columns = HP_DJ_Sto.columns[1:])
# weight_DF

In [16]:
# len(Portfolio[4])
weight = pd.DataFrame(Portfolio,
                      index = ['value','equal_weight','min_var','naive','tangent','large5','idustry_equal'], 
                      columns = HP_DJ_Sto.columns[1:-1])
                    #   ,index = ['value','equal_weight','min_var','naive','tangent','large5','idustry_equal'], columns = HP_DJ_Sto.columns[1:])
# HP_DJ_Sto_Ind
# largest_stocks_weights
# HP_DJ_Sto_Ind.index
# largest_stocks
# print(HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns)
# largest_stocks = HP_DJ_Sto_Ind.groupby(industries, axis=1).idxmax()
# print(largest_stocks.iloc[0,1])
# for stock in largest_stocks:
# # HP_DJ_Sto_Ind_ori.iloc[:,1:-1].columns:
#     print(stock)
weight

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DIS,...,MSFT,NKE,PG,CRM,BAC,UNH,VZ,HWP,PFE,WMT
value,0.058849,0.033593,0.035678,0.032092,0.033403,0.051938,0.061530,0.015495,0.033004,0.016666,...,0.018673,0.049526,0.035593,0.010507,0.055325,0.040699,0.023661,0.013795,0.022412,0.034495
equal_weight,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,...,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333
min_var,-0.054223,-0.019520,0.006407,0.000233,-0.047194,-0.025272,-0.032463,0.204430,0.022964,-0.036486,...,-0.092853,-0.004810,-0.103891,0.005730,-0.004574,-0.008195,0.098050,0.131386,0.513913,0.095976
naive,0.039922,0.024584,0.032634,0.024223,0.024914,0.024308,0.030633,0.030877,0.039531,0.032606,...,0.035466,0.025778,0.052461,0.018902,0.018718,0.029704,0.046836,0.025138,0.038681,0.046845
tangent,-0.071317,0.024424,0.058171,-0.001104,-0.065691,-0.047037,-0.036765,0.433695,-0.049539,-0.080224,...,-0.010301,0.002001,-0.239466,0.033192,0.015856,-0.010190,0.058627,0.220230,0.283659,0.055307
large5,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000
idustry_equal,0.020000,0.050000,0.020000,0.028571,0.050000,0.020000,0.050000,0.028571,0.020000,0.040000,...,0.028571,0.020000,0.020000,0.028571,0.050000,0.040000,0.050000,0.028571,0.020000,0.040000


2. Compare Portfolios

3.Performance Measures

In [27]:
RF = 0.02
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind_ori.copy()
HP_DJ_Sto_Ind = HP_DJ_Sto_Ind[4:]
HP_DJ_Sto_Ind.columns = HP_DJ_Sto_Ind.columns[:-1].tolist() + ['DJIA']
HP_DJ_Sto_Ind.columns =  ['Date'] +  HP_DJ_Sto_Ind.columns[1:].tolist()
Backtesting_Period = HP_DJ_Sto_Ind[HP_DJ_Sto_Ind['Date'] <= pd.to_datetime('2019-01-01')]
Test_period = HP_DJ_Sto_Ind[HP_DJ_Sto_Ind['Date'] > pd.to_datetime('2019-01-01')]


In [132]:
def Month_to_annu_return(price_list:pd.DataFrame):
    list_intern = []
    for i in range(0, price_list.shape[0]-12, 12):
        xx = price_list.iloc[i+11].values/price_list.iloc[i].values - 1
        list_intern.append(xx)
    return list_intern

def calculate_average_annualized_return(price_list: pd.DataFrame):
    """
    Calculate the average annualized return for each stock in the given DataFrame.

    Parameters:
    price_list (pd.DataFrame): DataFrame containing the stock prices.

    Returns:
    list: List containing the average annualized return for each stock.
    """
    # annu_return = Month_to_annu_return(price_list)
    # average_annu = np.mean(np.array(annu_return), axis=0).tolist()
    annu_return = np.mean(price_list.pct_change(),axis=0)*12
    average_annu = annu_return.tolist()
    # average_annu = annu_return.to_frame()
    return average_annu

def volatility(sto_price: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the volatility of the returns for each stock in the given DataFrame.

    Parameters:
    sto_price (pd.DataFrame): DataFrame containing the stock prices.

    Returns:
    pd.DataFrame: DataFrame containing the volatility of the returns for each stock.
    """
    # Calculate the daily returns
    daily_returns = sto_price.pct_change()
    
    # Calculate the volatility (standard deviation of daily returns)
    volatility = daily_returns.std() * np.sqrt(12)  # Assuming 252 trading days in a year
    
    return volatility

def skewness(sto_price: pd.DataFrame):
    daily_returns = sto_price.pct_change()
    skewness_values = daily_returns.skew()
    return skewness_values


def max_drawdown(sto_price: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the maximum drawdown for each stock in the given DataFrame.

    Parameters:
    sto_price (pd.DataFrame): DataFrame containing the stock prices.

    Returns:
    pd.DataFrame: DataFrame containing the maximum drawdown for each stock.
    """
    # Calculate the cumulative returns
    cumulative_returns = (sto_price.pct_change() + 1).cumprod()
    
    # Calculate the peak
    peak = cumulative_returns.expanding(min_periods=1).max()

    # Calculate the drawdown
    drawdown = (cumulative_returns - peak) / peak
    
    # Calculate the maximum drawdown
    max_drawdown = drawdown.min()
    
    return max_drawdown

def sharpe_ratio(sto_price: pd.DataFrame, RF :float):
    """
    Calculate the Sharpe Ratio for each stock in the given DataFrame.

    Parameters:
    sto_price (pd.DataFrame): DataFrame containing the monthly prices of each stock.
    RF (float): Risk-free rate.

    Returns:
    pd.DataFrame: DataFrame containing the Sharpe Ratio for each stock.
    """
    # Calculate the monthly returns
    monthly_returns = sto_price.pct_change()
    
    # Calculate the average monthly return
    avg_monthly_return = monthly_returns.mean() * 12
    
    # Calculate the standard deviation of monthly returns
    std_dev_monthly_return = monthly_returns.std() * np.sqrt(12)
    
    # Calculate the Sharpe Ratio
    sharpe_ratio = (avg_monthly_return - RF) / std_dev_monthly_return
    
    return sharpe_ratio

def calculate_portfolio_metrics(HP_DJ_Sto_Ind, RF):
    intern_list = []
    HP_DJ_Sto = HP_DJ_Sto_Ind.iloc[:,1:] #Data without date
    Date = HP_DJ_Sto_Ind.iloc[:,1] #Only Date

    average_annu = calculate_average_annualized_return(HP_DJ_Sto)
    intern_list.append(average_annu)
    
    volatility_values = volatility(HP_DJ_Sto).values.tolist()
    intern_list.append(volatility_values)
    
    skewness_values = skewness(HP_DJ_Sto).values.tolist()
    intern_list.append(skewness_values)
    
    max_drawdown_values = max_drawdown(HP_DJ_Sto).values.tolist()
    intern_list.append(max_drawdown_values)
    
    sharpe_ratio_values = sharpe_ratio(HP_DJ_Sto,RF).values.tolist()
    intern_list.append(sharpe_ratio_values)
    
    return intern_list

In [136]:
Risk_free = 0.02
BT_performance = calculate_portfolio_metrics(Backtesting_Period,Risk_free)
pp = pd.DataFrame(BT_performance,
                  index = ['Annu_return', 'Volatility','Skewness','Maximum drawdown','Sharpe Ratio'],
                   columns=HP_DJ_Sto_Ind.columns[1:])


C:\Users\zhaoj\AppData\Local\Temp\ipykernel_31572\480438297.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  annu_return = np.mean(price_list.pct_change(),axis=0)*12
C:\Users\zhaoj\AppData\Local\Temp\ipykernel_31572\480438297.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  daily_returns = sto_price.pct_change()
C:\Users\zhaoj\AppData\Local\Temp\ipykernel_31572\480438297.py:44: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead.

In [134]:
display(pp.T)

,Annu_return,Volatility,Skewness,Maximum drawdown,Sharpe Ratio
3M CO,0.069875,0.189747,-0.256305,-0.514212,0.262852
AMERICAN EXPRESS CO,0.091085,0.334060,3.518355,-0.814404,0.212791
AMGEN INC,0.116621,0.244815,0.946458,-0.483751,0.394671
APPLE COMPUTER INC,0.364020,0.336733,-0.226276,-0.569114,1.021641
BOEING CO,0.159160,0.249561,-0.380290,-0.700543,0.557618
CATERPILLAR INC,0.092488,0.335859,-0.649727,-0.747616,0.215829
CHEVRON CORP NEW,0.040038,0.232341,-1.772313,-0.466667,0.086242
CISCO SYSTEMS INC,0.074011,0.255110,0.059665,-0.560216,0.211717
COCA COLA CO,0.022966,0.206250,-4.429280,-0.551361,0.014380
DISNEY WALT CO,0.122530,0.207491,-0.202066,-0.526806,0.494142


In [135]:
# Filter the portfolio based on annual return and Sharpe Ratio
first_fliter = pp.loc[:, pp.loc['Annu_return'] >= pp.loc['Annu_return', 'DJIA']]
first_fliter = first_fliter.loc[:,first_fliter.loc['Sharpe Ratio'] >= first_fliter.loc['Sharpe Ratio','DJIA']]
# Display the shape of the original portfolio and the filtered portfolio
display(pp.shape)
display(first_fliter.shape)
# Display the filtered portfolio

display(first_fliter)

# display(first_fliter.sort_values(by='Sharpe Ratio', ascending=False)


(5, 31)

(5, 11)

,AMGEN INC,APPLE COMPUTER INC,BOEING CO,DISNEY WALT CO,HONEYWELL INTERNATIONAL INC,HOME DEPOT INC,MCDONALDS CORP,MICROSOFT CORP,SALESFORCE COM INC,UNITED HEALTHCARE CORP,DJIA
Annu_return,0.116621,0.364020,0.159160,0.122530,0.111631,0.132184,0.144342,0.113410,0.273080,0.140757,0.071071
Volatility,0.244815,0.336733,0.249561,0.207491,0.211285,0.211880,0.150045,0.228023,0.444459,0.280147,0.131181
Skewness,0.946458,-0.226276,-0.380290,-0.202066,-0.671818,-0.176363,-0.065711,0.180836,-1.104081,-1.950341,-0.687844
Maximum drawdown,-0.483751,-0.569114,-0.700543,-0.526806,-0.564236,-0.519880,-0.161125,-0.561261,-0.786501,-0.793982,-0.492970
Sharpe Ratio,0.394671,1.021641,0.557618,0.494142,0.433686,0.529470,0.828697,0.409650,0.569410,0.431048,0.389320


In [61]:
# import pandas as pd

# data = {'Return': [0.05, 0.10, 0.15], 'Risk': [0.02, 0.03, 0.04]}
# index = ['Annual_return', 'Quarterly_return', 'Monthly_return']
# df = pd.DataFrame(data, index=index)
# return_column = df['Return']
# display(df[0])
# display(df.loc['Annual_return', 'Return'])
# #loc[0]  ==  iloc['Annual_return]  == first row
# #column  ==  ['return]
# #unit    ==  df.loc['Annual_return', 'Return']